In [2]:
from ecosap.ecopro import SpectralComparison
from pathlib import Path
import sqlite3
import pandas as pd
import pickle

# Calculate PSD's

In [ ]:
conn = sqlite3.connect('../results/results.db')
cursor = conn.cursor()

unique_folders = pd.read_sql_query("SELECT DISTINCT folder FROM audios", conn)
unique_folders = unique_folders['folder'].tolist()

AUDIOPATH = Path('/Users/davidluna/Documents/Audios/compare-recorders/splitted/')

# GetPSD is a class that calculates the PSD of a signal
# and returns the PSD and the frequencies

for x, xfolder in enumerate(unique_folders):
    for y, yfolder in enumerate(unique_folders[x+1:]): 
        print(f'{xfolder}-{yfolder}')
        folder_x = pd.read_sql_query(f"SELECT path FROM audios WHERE folder = '{xfolder}'", conn)
        folder_x = folder_x['path'].tolist()
        folder_y = pd.read_sql_query(f"SELECT path FROM audios WHERE folder = '{yfolder}'", conn)
        folder_y = folder_y['path'].tolist()

        spec_df, f  = SpectralComparison().transform(folder_x, folder_y)
        # Save the PSDs in the database
        for ind,row in spec_df.iterrows():
            
            cursor.execute("INSERT INTO spectral_comparison (psd_x, psd_y, csd_xy, freq, folder_x, folder_y, n_audio) VALUES (?, ?, ?, ?, ?, ?, ?)", 
            (pickle.dumps(row.sxx), pickle.dumps(row.syy), 
            pickle.dumps(row.sxy), pickle.dumps(f), xfolder,
            yfolder,ind))
        conn.commit()    
conn.close()


# Calculate Coherence